# Environment configuration
**Note:** packages installer cell output is supressed. Remove the '%%capture' line in case of any problems with packages.

**Note:** to run this notebook in the google colab it is better to uncomment the next cell.

In [ ]:
# You need these lines in Google Colab

#from google.colab import output
#output.enable_custom_widget_manager()

In [1]:
"""
Magic capture supresses the output of the cell to prevent
long output of already satisfied dependencies.
Remove this line to enable cell output
"""
%%capture
%pip install ipympl

In [12]:
%matplotlib widget

In [4]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import matplotlib.patches as pltp
from matplotlib.animation import FuncAnimation, PillowWriter
from numpy import exp, pi, array, asarray
from sympy.solvers import solve
from sympy import Symbol, Ellipse, Point, Line, Matrix, diff

# Task 1

In [5]:
# use this variable to scale the graph
scale = 1

In [6]:
t = Symbol('t')
O = Matrix([3 * t, 4 * t**2 + 1])
V = diff(O, t)
a = diff(V, t)
at = a.dot(V) / V.norm() * V / V.norm()
an = a - at

In [7]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
xdata, ydata = [], []
ln, = plt.plot([], [], 'b-')

vecs = []

# animation initialisation
def init_task1():
    ax.set_xlim(-50 / scale, 50 / scale)
    ax.set_ylim(-1 / scale, 105 / scale)
    plt.xlabel('x')
    plt.ylabel('y(x)')
    plt.grid()
    plt.title('Scale: {}'.format(scale))
    return ln,

# animation update on every frame
def update_task1(frame):
    # substitute frame for time variable
    # in sympy objects
    Of = O.subs(t, frame)
    Vf = V.subs(t, frame)
    af = a.subs(t, frame)
    atf = at.subs(t, frame)
    anf = an.subs(t, frame)

    # remove all the vectors on the graph
    while len(vecs):
        vecs[-1].remove()
        vecs.pop()

    # draw new vectors at the corresponding point
    vecs.extend([
        plt.quiver([float(Of[0])], [float(Of[1])], [float(i[0])], [float(i[1])], units='xy', scale = 1, scale_units='xy', angles='xy', color=color)
        for i, color in zip([Vf, af, atf, anf], ['black', 'green', 'red', 'blue'])
    ])

    # draw new point on the graph
    xdata.append(Of[0])
    ydata.append(Of[1])
    ln.set_data(xdata, ydata)
    return ln,

# create animation
anim = FuncAnimation(fig, update_task1, frames=np.linspace(-5, 5, 120), init_func=init_task1, blit=True)

# save animation to the file
anim.save('task1.gif', dpi=100, writer=PillowWriter(fps=60))
# prevent unclosed plots
plt.close('all')

# Task 2

In [8]:
"""
Custom method for rotating a list of vectors
with respect to some origin on a given angle.
Angle should be given in radians.
"""
def c_rot(vectors, angle, origin = [0, 0]):
        for i in range(len(vectors)):
            vectors[i] = complex(vectors[i][0] - origin[0], vectors[i][1] - origin[1])
            vectors[i] *= exp(angle * complex(0, 1))
            vectors[i] = [vectors[i].real + origin[0], vectors[i].imag + origin[1]]
        return vectors

In [9]:
# Initial values
OA = OP = 25
AC = 20
w = 1

In [10]:
t = Symbol('t')
O = array([0, 0])
P = array([0, OP])
P1 = c_rot([P], pi / 3)[0]
A = OA * Point([sp.cos(t), sp.sin(t)])
B = Ellipse(A, 80, 80).intersection(Line(Point(P), Point(P1)))[0]
C = 1 / 4 * (B - A) + A
Pabc = [[A[0], A[1]], [B[0], B[1]], [C[0], C[1]]]
Vabc = [[sp.diff(i[0]), sp.diff(i[1])] for i in Pabc]
Aabc = [[sp.diff(i[0]), sp.diff(i[1])] for i in Vabc]

In [11]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.axis("off")

xdata, ydata = [], []
ln, = plt.plot([], [], 'b-')

vectors = []
points  = []
titles  = []
lines   = []

# Animation initialisation
def init_model():
    ax.set_xlim(-130, 130)
    ax.set_ylim(-130, 130)
    plt.title('Model')
    return ln,

# Draw a rectangle at a given point   
def plot_poly(Bt):
    h = 10
    w = 20
    poly = pltp.Polygon(c_rot([[Bt[0] - w, Bt[1] - h],
                               [Bt[0] - w, Bt[1] + h],
                               [Bt[0] + w, Bt[1] + h], 
                               [Bt[0] + w, Bt[1] - h]], pi / 6, Bt), 
                        fill=False, edgecolor="blue", linewidth=0.5)
    ax.add_patch(poly)

# Remove all the previous points, labels, vectors   
def clear_screen():
    if len(ax.patches):
        ax.patches.pop()
    
    while len(points):
        for item in points[-1]:
            item.remove()
        points.pop()
    
    while len(lines):
        for item in lines[-1]:
            item.remove()
        lines.pop()
        
    while len(titles):
        titles[-1].remove()
        titles.pop()
        
    while len(vectors):
        vectors[-1].remove()
        vectors.pop()

# Animation update on each frame
def update_model(frame):
    # Substitute frame for t in sympy objects
    PFabc = [[float(i[0].subs(t, frame).evalf()), float(i[1].subs(t, frame).evalf())] for i in Pabc]
    VFabc = [[float(i[0].subs(t, frame).evalf()), float(i[1].subs(t, frame).evalf())] for i in Vabc]            
    AFabc = [[float(i[0].subs(t, frame).evalf()), float(i[1].subs(t, frame).evalf())] for i in Aabc]
    
    Tabc = [VFabc[i] / np.linalg.norm(VFabc[i]) for i in range(3)]
    
    ATabc = [Tabc[i] * (np.dot(AFabc[i], VFabc[i]) / np.linalg.norm(VFabc[i])) for i in range(3)]
    ANabc = [np.asarray(AFabc[i]) - np.asarray(ATabc[i]) for i in range(3)]
    
    Pf = P
    Of = O
    Cf = C
    
    
    clear_screen()    
    plot_poly(PFabc[1])

    # Draw all points of interest on the screen    
    points.extend([
        plt.plot(i[0], i[1], marker="o", markersize=3, markeredgecolor="black", markerfacecolor="black")
        for i in [PFabc[0], PFabc[1], PFabc[2], Of, Pf, P1]
    ])
    
    # Plot all lines on the screen
    lines.extend([
        plt.plot([Pf[0], PFabc[1][0]], [Pf[1], PFabc[1][1]], linestyle="dashed", linewidth=0.5, color="green"),
        plt.plot([Pf[0], Of[0]], [Pf[1], Of[1]], linestyle="dashed", linewidth=0.5, color="blue"),
        plt.plot([PFabc[0][0], PFabc[1][0]], [PFabc[0][1], PFabc[1][1]], linewidth=0.5, color="black"),
        plt.plot([Of[0], PFabc[0][0]], [Of[1], PFabc[0][1]], linewidth=0.5, color="black")
    ])
    
    # Add titles to points on the plot
    titles.extend([
        ax.text(i[0][0], i[0][1] + 3, i[1], fontsize=10)
        for i in [[PFabc[0], 'A'], [PFabc[1], 'B'], [PFabc[2], 'C'], [Of, 'O'], [Pf, 'P'], [P1, r'$P_1$']]
    ])
    
    # Draw all the velocity vectors on the plot
    vectors.extend([
        plt.quiver([i[0]], [i[1]], [j[0]], [j[1]], color="black", units='xy', scale = 1, scale_units='xy', angles='xy')
        for i, j in zip(PFabc, VFabc)
    ])
    
    # Draw all the total acceleration vectors on the plot
    vectors.extend([
        plt.quiver([i[0]], [i[1]], [j[0]], [j[1]], color="green", units='xy', scale = 1, scale_units='xy', angles='xy')
        for i, j in zip(PFabc, AFabc)
    ])
    
    # Draw all the tangential acceleration vectors on the plot
    vectors.extend([
        plt.quiver([i[0]], [i[1]], [j[0]], [j[1]], color="red", units='xy', scale = 1, scale_units='xy', angles='xy')
        for i, j in zip(PFabc, ATabc)
    ])
    
    # Draw all the normal acceleration vectors on the plot
    vectors.extend([
        plt.quiver([i[0]], [i[1]], [j[0]], [j[1]], color="blue", units='xy', scale = 1, scale_units='xy', angles='xy')
        for i, j in zip([PFabc[0], PFabc[2]], [ANabc[0], ANabc[2]])
    ])
    
    return ln,

anim = FuncAnimation(fig, update_model, frames=np.linspace(0, 2 * np.pi, 120),
                        init_func=init_model, blit=True)

anim.save('task2.gif', dpi=300, writer=PillowWriter(fps=60))
plt.close('all')